In [12]:
# Emotion Detection Preprocessing Notebook
# ResNet-18 Architecture for 48x48 Grayscale Images

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("Environment Setup Complete!")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Environment Setup Complete!
PyTorch Version: 2.1.0+cu121
CUDA Available: False


In [14]:
# DIRECTORY CONFIGURATION 

CONFIG = {
    # Main dataset directory
    'dataset_root': '../../data/raw/fer2013/train',
    
    'train_dir': '../../data/raw/fer2013/train',
    'test_dir': '../../data/raw/fer2013/test',

# Output directories
    'output_dir': '../../data/processed_data',
    'model_save_dir': '../../models',
    
    # Dataset parameters (since images are already 48x48 grayscale)
    'image_size': (48, 48),
    'batch_size': 32,
    'validation_split': 0.2,
    'test_split': 0.1,
    
    # Training parameters
    'num_epochs': 50,
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    'patience': 10,
    
    # Emotion labels (modify according to your dataset)
    'emotion_labels': ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
}


In [15]:
# Create output directories
for dir_path in [CONFIG['output_dir'], CONFIG['model_save_dir']]:
    os.makedirs(dir_path, exist_ok=True)

print("Configuration loaded successfully!")

Configuration loaded successfully!


In [16]:
# =============================================================================
# EFFICIENT DATASET CLASS (No EDA)
# =============================================================================

class EmotionDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_encoder = LabelEncoder()
        
        # Load images from directory structure
        for emotion in CONFIG['emotion_labels']:
            emotion_dir = os.path.join(data_dir, emotion)
            if os.path.exists(emotion_dir):
                for img_file in os.listdir(emotion_dir):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(emotion_dir, img_file)
                        self.data.append(img_path)
                        self.labels.append(emotion)
        
        # Encode labels
        self.labels = self.label_encoder.fit_transform(self.labels)
        print(f"Loaded {len(self.data)} images across {len(self.label_encoder.classes_)} classes")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        
        # Load image (already grayscale 48x48)
        image = Image.open(img_path)
        
        # Safety checks (minimal since images are pre-processed)
        if image.mode != 'L':
            image = image.convert('L')
        if image.size != CONFIG['image_size']:
            image = image.resize(CONFIG['image_size'])
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [17]:
# =============================================================================
# OPTIMIZED TRANSFORMS (No Resize Needed)
# =============================================================================

def get_transforms():
    """Optimized transforms for pre-processed 48x48 grayscale images"""
    
    # Training transforms with augmentation
    train_transforms = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
    
    # Validation/Test transforms (minimal processing)
    val_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
    
    return train_transforms, val_transforms

In [18]:
# =============================================================================
# RESNET-18 MODEL FOR GRAYSCALE EMOTION DETECTION
# =============================================================================

class EmotionResNet(nn.Module):
    def __init__(self, num_classes=7, pretrained=True):
        super(EmotionResNet, self).__init__()
        
        # Load pretrained ResNet-18
        self.resnet = models.resnet18(pretrained=pretrained)
        
        # Modify first layer for grayscale input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify final layer for emotion classification
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
        # Initialize the new conv1 layer
        nn.init.kaiming_normal_(self.resnet.conv1.weight, mode='fan_out', nonlinearity='relu')
    
    def forward(self, x):
        return self.resnet(x)

In [19]:
# =============================================================================
# TRAINING SETUP FUNCTIONS
# =============================================================================

def setup_training(model, device):
    """Setup training components"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=CONFIG['learning_rate'],
        weight_decay=CONFIG['weight_decay']
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=CONFIG['patience']//2, verbose=True
    )
    return criterion, optimizer, scheduler

def calculate_dataset_statistics(data_loader):
    """Calculate actual mean and std of your dataset"""
    print("Calculating dataset statistics...")
    
    mean = torch.zeros(1)
    std = torch.zeros(1)
    total_samples = 0
    
    for images, _ in data_loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_samples += batch_samples
    
    mean /= total_samples
    std /= total_samples
    
    print(f"Your dataset - Mean: {mean.item():.4f}, Std: {std.item():.4f}")
    return mean.item(), std.item()


In [20]:
# =============================================================================
# MAIN PREPROCESSING PIPELINE
# =============================================================================

def create_data_loaders():
    """Create train/validation/test data loaders"""
    
    print("Creating data loaders...")
    
    # Load dataset
    dataset = EmotionDataset(CONFIG['dataset_root'])
    
    # Create data splits
    train_idx, temp_idx = train_test_split(
        range(len(dataset)), 
        test_size=CONFIG['validation_split'] + CONFIG['test_split'],
        stratify=dataset.labels, random_state=42
    )
    
    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=CONFIG['test_split'] / (CONFIG['validation_split'] + CONFIG['test_split']),
        stratify=[dataset.labels[i] for i in temp_idx], random_state=42
    )
    
    print(f"Data splits - Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")
    
    # Get transforms
    train_transforms, val_transforms = get_transforms()
    
    # Create subset datasets
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)
    test_dataset = torch.utils.data.Subset(dataset, test_idx)
    
    # Apply transforms
    train_dataset.dataset.transform = train_transforms
    val_dataset.dataset.transform = val_transforms
    test_dataset.dataset.transform = val_transforms
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], 
                            shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], 
                          shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], 
                           shuffle=False, num_workers=4, pin_memory=True)
    
    return train_loader, val_loader, test_loader, dataset

def initialize_model():
    """Initialize the ResNet-18 model"""
    
    print("Initializing model...")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = EmotionResNet(num_classes=len(CONFIG['emotion_labels']))
    model = model.to(device)
    
    # Model summary
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    return model, device

def save_preprocessing_info(dataset, train_size, val_size, test_size):
    """Save preprocessing information"""
    
    import json
    
    preprocessing_info = {
        'dataset_size': len(dataset),
        'train_size': train_size,
        'val_size': val_size,
        'test_size': test_size,
        'num_classes': len(CONFIG['emotion_labels']),
        'emotion_labels': CONFIG['emotion_labels'],
        'image_size': CONFIG['image_size'],
        'batch_size': CONFIG['batch_size'],
        'label_encoder_classes': dataset.label_encoder.classes_.tolist(),
    }
    
    with open(os.path.join(CONFIG['output_dir'], 'preprocessing_info.json'), 'w') as f:
        json.dump(preprocessing_info, f, indent=2)
    
    print(f"Preprocessing info saved to: {os.path.join(CONFIG['output_dir'], 'preprocessing_info.json')}")
    return preprocessing_info

In [21]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main preprocessing pipeline execution"""
    
    print("="*50)
    print("EMOTION DETECTION PREPROCESSING")
    print("="*50)
    
    # Create data loaders
    train_loader, val_loader, test_loader, dataset = create_data_loaders()
    
    # Initialize model
    model, device = initialize_model()
    
    # Setup training components
    criterion, optimizer, scheduler = setup_training(model, device)
    
    # Calculate dataset statistics (optional - for fine-tuning normalization)
    print("\nCalculating dataset statistics...")
    temp_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
    mean, std = calculate_dataset_statistics(temp_loader)
    print(f"Consider updating normalization to: mean=[{mean:.4f}], std=[{std:.4f}]")
    
    # Save preprocessing information
    save_preprocessing_info(dataset, len(train_loader.dataset), 
                          len(val_loader.dataset), len(test_loader.dataset))
    
    print("\nPreprocessing completed successfully!")
    print("Ready for training!")
    
    return {
        'train_loader': train_loader,
        'val_loader': val_loader,
        'test_loader': test_loader,
        'model': model,
        'device': device,
        'criterion': criterion,
        'optimizer': optimizer,
        'scheduler': scheduler,
        'dataset_stats': {'mean': mean, 'std': std}
    }

In [ ]:
# =============================================================================
# USAGE EXAMPLE
# =============================================================================

if __name__ == "__main__":
    
    
    results = main()
    
    # Example usage after running main():
    """
    # Get the preprocessed components
    train_loader = results['train_loader']
    val_loader = results['val_loader']
    test_loader = results['test_loader']
    model = results['model']
    device = results['device']
    criterion = results['criterion']
    optimizer = results['optimizer']
    scheduler = results['scheduler']
    
    # Ready for training loop!
    # Your training code goes here...
    """

EMOTION DETECTION PREPROCESSING
Creating data loaders...
Loaded 28709 images across 7 classes
Data splits - Train: 20096, Val: 5742, Test: 2871
Initializing model...
Using device: cpu


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Total parameters: 11,436,487
Trainable parameters: 11,436,487

Calculating dataset statistics...
Calculating dataset statistics...
